In [3]:
! pip install tika

  Created wheel for tika: filename=tika-1.24-py3-none-any.whl size=32883 sha256=f93cfb66b7ca3f30cad0e26841018729c027ea1b5a622fc16786a1136d494b61
  Stored in directory: /home/jovyan/.cache/pip/wheels/ec/2b/38/58ff05467a742e32f67f5d0de048fa046e764e2fbb25ac93f3
Successfully built tika


In [4]:
from tika import unpack # for extracting text from PDFs

In [12]:
import os 
import re

In [6]:
def pdf2text(textkey):
    PDF = unpack.from_file(os.path.join('..','papers',f'{textkey}.pdf'))
    text = PDF['content']
    return text

In [7]:
textkey = 'ASingletonSpielman2014'

In [8]:
text = pdf2text(textkey)

2020-11-26 12:07:07,058 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2020-11-26 12:07:18,758 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2020-11-26 12:07:19,447 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [10]:
def get_refs(text, final_full_stop=True):
    
    if final_full_stop:
        refs = re.findall(r"[A-Z_-].+(?:19|20)[\d][\d].*(?:[\n]|.)+?[.][\n]", text)
    else:
        refs = re.findall(r"[A-Z_-].+(?:19|20)[\d][\d].*(?:[\n]|.)+?[\n]", text)
    
    return refs

In [13]:
refs = get_refs(text)

In [15]:
def tidy_refs(refs):
    
    tidy_refs = []
    for ref in refs:
        while '\n' in ref:
            ref = ref.replace('\n',' ')
        while '  '  in ref:
            ref = ref.replace('  ',' ')
            
        tidy_refs.append(ref)
        
    return tidy_refs

In [17]:
tidy_refs = tidy_refs(refs)

In [20]:
def simple_parse(ref):
    
    parsed = {}
    parsed['original'] = ref
    try:
        date = re.findall(r"(?:19|20)[\d][\d]", ref)[0]
    except:
        date = 'Unknown'
    parsed['date'] = date
    names = ref.split(date)[0]
    surnames = re.findall(r"[A-ZÀ-ÖØ][A-Za-zÀ-ÖØ-öø-ÿ]+?[,]", names)
    
    for i, surname in enumerate(surnames):
        names = names.replace(surname,'')
        surname = surname.lower().capitalize()
        surnames[i] = surname.replace(',','')
        
    initials = re.findall(r"[A-Z]", names)
    parsed['initials'] = initials   
    parsed['surnames'] = surnames
    try:
        parsed['etc'] = re.findall(r"[A-ZÀ-ÖØ][A-Za-z].+", ref.split(date)[1])[0]
    except:
        parsed['etc'] = ''
        split = parsed['etc'].split('.')
        parsed['split'] = split
        parsed['title'] = split[0]
    
    try:
        textkey = initials[0] + surnames[0]
        if len(surnames) < 3:
            if len(surnames) > 1:
                textkey += surnames[1]
        else:
            textkey += 'EtAl'
        textkey += date
        parsed['textkey'] = textkey
    except:
        pass
       
    return parsed

In [21]:
parsed = simple_parse(tidy_refs)

AttributeError: 'list' object has no attribute 'split'

In [23]:
class Text():
    
    def __init__(self, textkey=None, pdftext=None, harvard_ref=None):
        
        self.key = textkey
         
        if pdftext:
            self.text = pdftext
        elif textkey:
            self.text = pdf2text(textkey)
        else:
            self.text = None
        
        self.harvard_ref = harvard_ref
        if self.harvard_ref:
            parse = simple_parse(self.harvard_ref)
            if not self.key:
                try:
                    self.key = parse['textkey']
                except:
                    pass
       
        self.refs = None
        self.links = None
        
    def __repr__(self):
        
        if self.key:
            return self.key
        else:
            return self.harvard_ref
    
    def get_refs(self, text=None):
        
        if text:       
            self.refs = tidy_refs(get_refs(text))
        elif self.text:
            self.refs = tidy_refs(get_refs(self.text))
        else:
            print('Need text to get refs.')
 

In [24]:
t = Text(textkey)

In [34]:
get_refs(t.text)

['On: 09 December 2014, At: 18:19\nPublisher: Routledge\nInforma Ltd Registered in England and Wales Registered Number: 1072954 Registered\noffice: Mortimer House, 37-41 Mortimer Street, London W1T 3JH, UK\n\nThe Professional Geographer\nPublication details, including instructions for authors and\nsubscription information:\nhttp://www.tandfonline.com/loi/rtpg20\n\nThe Past, Present, and Future of\nGeodemographic Research in the United\nStates and United Kingdom\nAlexander D. Singletona & Seth E. Spielmanb\na University of Liverpool\nb University of Colorado at Boulder\nPublished online: 27 Nov 2013.\n',
 'To cite this article: Alexander D. Singleton & Seth E. Spielman (2014) The Past, Present, and Future\nof Geodemographic Research in the United States and United Kingdom, The Professional Geographer,\n66:4, 558-567, DOI: 10.1080/00330124.2013.848764\n\nTo link to this article:  http://dx.doi.org/10.1080/00330124.2013.848764\n\nPLEASE SCROLL DOWN FOR ARTICLE\n\nTaylor & Francis makes ev